In [ ]:
import os
import re
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ============ 1. 读入各种数据表/路径配置 ============
base_dir = "../DATA/Labeled_data/UsingValidationData"

# 1) 读取 "processed_results.csv"：单文件夹粒度的真实准确率/延迟/jitter
#    columns: folder_name, model_name, accuracy, delayed_time, jitter_count, ...
df_folders = pd.read_csv(os.path.join(base_dir, "processed_results.csv"))

# 2) 读取 "model_stats.csv"：同模型的平均指标（可以是你之前统计好的 DataFrame）
#    index = model_name, columns = [accuracy, delayed_time, jitter_count]
df_model_avg = pd.read_csv(os.path.join(base_dir, "model_performance_summary.csv"), index_col="model_name")

# 3) 预先设定我们想要的手势类型（如果想固定顺序）
gestures_of_interest = ["ST", "G", "P"]


# ============ 2. 扫描子文件夹，解析 model 与 gesture，并读取 CSV ============
# data_dict[model_name][gesture] = {
#     "df": 该文件夹里的 time 序列数据,
#     "folder_name": 该文件夹名,
#     "acc": 该文件夹对应的准确率(单次),
#     "delay": ...,
#     "jitter": ...
# }
data_dict = {}

# 利用正则，匹配：
#   0214_NCPCfC_Post123_T4ES_G
#   group(1) = NCPCfC, group(2) = G
pattern = r"^[^_]+_([^_]+)_.*_([^_]+)$"

for folder in os.listdir(base_dir):
    full_path = os.path.join(base_dir, folder)
    if not os.path.isdir(full_path):
        continue
    
    match = re.match(pattern, folder)
    if not match:
        continue
    
    model_name = match.group(1)
    gesture_name = match.group(2)
    
    # 可选：只关心特定手势
    if gesture_name not in gestures_of_interest:
        continue
    
    csv_file = os.path.join(full_path, "labeled_modelResult_data.csv")
    if not os.path.isfile(csv_file):
        print(f"警告：{csv_file} 不存在，跳过。")
        continue
    
    # 读取该单次CSV
    df_data = pd.read_csv(csv_file)
    
    # 在 processed_results.csv 里找到这一行
    df_folder_stats = df_folders[df_folders["folder_name"] == folder]
    if len(df_folder_stats) == 0:
        print(f"警告：{folder} 不在 processed_results.csv 里，跳过。")
        continue
    
    # 取这一行的单次 accuracy / delay / jitter
    # 如果1个folder可能对应多行，就只取第一行
    single_acc    = df_folder_stats["accuracy"].iloc[0]
    single_delay  = df_folder_stats["delayed_time"].iloc[0]
    single_jitter = df_folder_stats["jitter_count"].iloc[0]
    
    # 保存到 data_dict
    if model_name not in data_dict:
        data_dict[model_name] = {}
    data_dict[model_name][gesture_name] = {
        "df": df_data,
        "folder_name": folder,
        "acc": single_acc,
        "delay": single_delay,
        "jitter": single_jitter
    }

# ============ 3. 确定子图行列顺序 ============
# 行：模型（顺序按照 df_model_avg.index 来排，且只保留在 data_dict 里能找到的）
all_models = [m for m in df_model_avg.index if m in data_dict]
# 列：手势
all_gestures = gestures_of_interest

num_rows = len(all_models)
num_cols = len(all_gestures)

# ============ 4. 创建子图并布局 ============
# 调大 vertical_spacing，让行间更宽敞
fig = make_subplots(
    rows=num_rows, cols=num_cols,
    subplot_titles=[f"{m} - {g}" for m in all_models for g in all_gestures],
    vertical_spacing=0.15 / num_rows,  # 适当放大行间距
    horizontal_spacing=0.06 / num_cols
)

# ============ 5. 循环添加曲线 & 单次指标标注 ============
for i, model_name in enumerate(all_models):
    for j, gesture_name in enumerate(all_gestures):
        row_idx = i + 1
        col_idx = j + 1
        
        info = data_dict.get(model_name, {}).get(gesture_name, None)
        if info is None:
            # 没有该手势数据就跳过
            continue
        
        df_data = info["df"]
        # 画你要的曲线，比如 touch_type_idx & label
        x_data = df_data["time"]
        y1 = df_data["touch_type_idx"]
        y2 = df_data["label"]
        
        # 给两条曲线不同颜色
        fig.add_trace(
            go.Scatter(x=x_data, y=y1, mode="lines", name="touch_type_idx", 
                       line=dict(color="red")),  # 根据喜好调颜色
            row=row_idx, col=col_idx
        )
        fig.add_trace(
            go.Scatter(x=x_data, y=y2, mode="lines", name="label",
                       line=dict(color="blue")),
            row=row_idx, col=col_idx
        )
        
        # 在子图上写“单次准确率”
        acc_1  = info["acc"]
        dly_1  = info["delay"]
        jit_1  = info["jitter"]
        text_1 = f"Acc={acc_1:.3f}<br>Delay={dly_1:.3f}<br>Jitter={jit_1:.1f}"
        
        # 放在子图的左上角（x=0.02,y=0.80等），可自行微调
        fig.add_annotation(
            text=text_1,
            x=0.02,
            y=0.85,
            xref=f"x{row_idx}{col_idx} domain",
            yref=f"y{row_idx}{col_idx} domain",
            showarrow=False,
            font=dict(size=12, color="black"),
            row=row_idx, col=col_idx
        )

# ============ 6. 在每一行下方标注“模型平均指标” ============
# 我们可以利用“domain”取到某一行在 y 方向的范围，然后把文字加到它下面一点
# 下面写一个循环，每行给它加注释
for i, model_name in enumerate(all_models):
    # 先从 df_model_avg 里取
    avg_acc    = df_model_avg.loc[model_name, "accuracy"]
    avg_delay  = df_model_avg.loc[model_name, "delayed_time"]
    avg_jitter = df_model_avg.loc[model_name, "jitter_count"]
    
    avg_text = (f"【{model_name}平均】Acc={avg_acc:.3f}, "
                f"Delay={avg_delay:.3f}, Jitter={avg_jitter:.1f}")
    
    # y 轴 domain （上界, 下界）= fig.layout["yaxis{...}"].domain
    # 该行对应的是 yaxis{i*num_cols + 1}，因为 col=1
    # 例如：row 1 => yaxis1; row 2 => yaxis4; row 3 => yaxis7; ...
    # 也可用 fig._subplot_ref(row=i+1, col=1) 拿到 yaxis编号
    # 这里做一个简单方法：我们知道 row i+1 => subplot_id = (i)*num_cols+1
    subplot_id = (i) * num_cols + 1
    yaxis_name = "yaxis" + (str(subplot_id) if subplot_id > 1 else "")
    
    if yaxis_name not in fig.layout:
        continue
    
    # domain 是一个形如 [0.694, 0.899] 的列表 (下界, 上界)
    y_top = fig.layout[yaxis_name].domain[1]
    y_bottom = fig.layout[yaxis_name].domain[0]
    
    # 我们想把平均值标在这一行的下方一点，可以 y = (y_bottom - 一点点)
    # 防止超出图表区，可以再加大 figure 的 margin 或者设 negative spacing
    y_annot = y_bottom - 0.04  # 你可以调大小
    
    fig.add_annotation(
        text=avg_text,
        x=0.5,  # 居中
        y=y_annot,
        xref="paper",
        yref="paper",  # 以整个图的 paper 坐标系来定位
        showarrow=False,
        font=dict(size=13, color="blue"),
        align="center"
    )

# ============ 7. 调整整体布局并显示 ============
# 可加大 overall margin，免得底部文字被裁剪
fig.update_layout(
    height=300 * num_rows + 100,  # 行多时可加大
    width=400 * num_cols,
    margin=dict(t=60, b=100, l=40, r=40),  # 上下左右边距
    title_text="各模型 × 各手势 数据对比",
    hovermode="x unified"
)

fig.show()


FileNotFoundError: [Errno 2] No such file or directory: '../DATA/Labeled_data/UsingValidationData/model_stats.csv'